![sql](img/sql-logo.jpg)

# A guide to databases, SQL, & using them with `pandas`

Let's start with Peter Bell's great blog piece [on relational databases](https://flatironschool.com/blog/an-introduction-to-the-relational-database).

Themes:
- Structured vs. Unstructured Data
- Records (Rows) in Tables in Databases
- Eliminating Redundancy

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Group Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

This sort of picture is sometimes called an **E**ntity **R**elationship **D**iagram. [Here](https://www.smartdraw.com/entity-relationship-diagram/) is another good resource for explaining typical ERD conventions.

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [10]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute(
"""
SELECT name
FROM sqlite_master
WHERE type='table';
"""
)
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


For more on "sqlite_master", see [here](https://www.techonthenet.com/sqlite/sys_tables/index.php).

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [4]:
cursor.execute(
"""
SELECT *
FROM airports
"""
)

#### Use description

In [5]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

The septuple structure has to do with Python API [compatibility issues](https://kite.com/python/docs/sqlite3.Cursor.description).

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`. The latter takes a number of records to return as a parameter.

In [9]:
cursor.fetchmany(10)

[(30,
  '31',
  'Brandon Muni',
  'Brandon',
  'Canada',
  'YBR',
  'CYBR',
  '49.91',
  '-99.951944',
  '1343',
  '-6',
  'A',
  'America/Winnipeg'),
 (31,
  '32',
  'Cambridge Bay',
  'Cambridge Bay',
  'Canada',
  'YCB',
  'CYCB',
  '69.108055',
  '-105.138333',
  '90',
  '-7',
  'A',
  'America/Edmonton'),
 (32,
  '33',
  'Nanaimo',
  'Nanaimo',
  'Canada',
  'YCD',
  'CYCD',
  '49.052333',
  '-123.870167',
  '93',
  '-8',
  'A',
  'America/Vancouver'),
 (33,
  '34',
  'Castlegar',
  'Castlegar',
  'Canada',
  'YCG',
  'CYCG',
  '49.296389',
  '-117.6325',
  '1624',
  '-8',
  'A',
  'America/Vancouver'),
 (34,
  '35',
  'Miramichi',
  'Chatham',
  'Canada',
  'YCH',
  'CYCH',
  '47.007778',
  '-65.449167',
  '108',
  '-4',
  'A',
  'America/Halifax'),
 (35,
  '36',
  'Charlo',
  'Charlo',
  'Canada',
  'YCL',
  'CYCL',
  '47.990833',
  '-66.330278',
  '132',
  '-4',
  'A',
  'America/Halifax'),
 (36,
  '37',
  'Kugluktuk',
  'Coppermine',
  'Canada',
  'YCO',
  'CYCO',
  '67.816667

**Task:** Get the descriptive data for airlines and routes tables

In [12]:
cursor.execute(
"""
SELECT *
FROM airlines
"""
).description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('alias', None, None, None, None, None, None),
 ('iata', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('callsign', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('active', None, None, None, None, None, None))

In [45]:
cursor.execute(
"""
SELECT *
FROM airports

"""
).description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

In [35]:
cursor.execute(
"""
SELECT *
FROM routes

"""
).description

(('index', None, None, None, None, None, None),
 ('airline', None, None, None, None, None, None),
 ('airline_id', None, None, None, None, None, None),
 ('source', None, None, None, None, None, None),
 ('source_id', None, None, None, None, None, None),
 ('dest', None, None, None, None, None, None),
 ('dest_id', None, None, None, None, None, None),
 ('codeshare', None, None, None, None, None, None),
 ('stops', None, None, None, None, None, None),
 ('equipment', None, None, None, None, None, None))

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**SELECT**

**FROM**

**WHERE**

**GROUP BY**

**ORDER BY**

**LIMIT**

#### Options for each:

**Select**:  `DISTINCT`, using `AS` to rename columns, single number aggregates, `COUNT()` to count, `*` for "all"

**From:** also uses aliasing with `AS`

**Where**: `=`, `BETWEEN`, `IN`, wildcards with `%`, `AND`, `OR`, `NOT` pattern matching with `LIKE` and `ILIKE`

**Order by**: `ASC` and `DESC`

**Limit**:  #

***

**Task**: 
- Select only active airlines in the United Kingdom from the airlines table <br/>
Hint: The values for 'active' are 'Y' and 'N'.
- Select the unique list of countries with airports

In [23]:
cursor.execute("""
SELECT * 
FROM airlines
WHERE active like 'Y' and country like 'United Kingdom'""").fetchall()

[(111,
  '112',
  'Astraeus',
  '\\N',
  '5W',
  'AEU',
  'FLYSTAR',
  'United Kingdom',
  'Y'),
 (491,
  '492',
  'Air Southwest',
  '\\N',
  None,
  'WOW',
  'SWALLOW',
  'United Kingdom',
  'Y'),
 (507,
  '508',
  'Aurigny Air Services',
  '\\N',
  'GR',
  'AUR',
  'AYLINE',
  'United Kingdom',
  'Y'),
 (564,
  '565',
  'Air Wales',
  '\\N',
  '6G',
  'AWW',
  'RED DRAGON',
  'United Kingdom',
  'Y'),
 (664,
  '665',
  'AD Aviation',
  '\\N',
  None,
  'VUE',
  'FLIGHTVUE',
  'United Kingdom',
  'Y'),
 (689, '690', 'Air Foyle', '\\N', 'GS', 'UPA', 'FOYLE', 'United Kingdom', 'Y'),
 (1353,
  '1355',
  'British Airways',
  '\\N',
  'BA',
  'BAW',
  'SPEEDBIRD',
  'United Kingdom',
  'Y'),
 (1409,
  '1411',
  'British International Helicopters',
  '\\N',
  'BS',
  'BIH',
  'BRINTEL',
  'United Kingdom',
  'Y'),
 (1435,
  '1437',
  'bmi',
  'bmi British Midland',
  'BD',
  'BMA',
  'MIDLAND',
  'United Kingdom',
  'Y'),
 (1439, '1441', 'bmibaby', '\\N', 'WW', 'BMI', 'BABY', 'United Kingd

In [20]:
cursor.execute("""
SELECT DISTINCT(country) 
FROM airports""").fetchall()

[('Papua New Guinea',),
 ('Greenland',),
 ('Iceland',),
 ('Canada',),
 ('Algeria',),
 ('Benin',),
 ('Burkina Faso',),
 ('Ghana',),
 ("Cote d'Ivoire",),
 ('Nigeria',),
 ('Niger',),
 ('Tunisia',),
 ('Togo',),
 ('Belgium',),
 ('Germany',),
 ('United States',),
 ('Estonia',),
 ('Finland',),
 ('United Kingdom',),
 ('Guernsey',),
 ('Jersey',),
 ('Isle of Man',),
 ('Falkland Islands',),
 ('Netherlands',),
 ('Ireland',),
 ('Denmark',),
 ('Faroe Islands',),
 ('Luxembourg',),
 ('Norway',),
 ('Poland',),
 ('Sweden',),
 ('South Africa',),
 ('Botswana',),
 ('Congo (Brazzaville)',),
 ('Congo (Kinshasa)',),
 ('Swaziland',),
 ('Central African Republic',),
 ('Equatorial Guinea',),
 ('Saint Helena',),
 ('Mauritius',),
 ('British Indian Ocean Territory',),
 ('Cameroon',),
 ('Zambia',),
 ('Comoros',),
 ('Mayotte',),
 ('Reunion',),
 ('Madagascar',),
 ('Angola',),
 ('Puerto Rico',),
 ('Gabon',),
 ('Sao Tome and Principe',),
 ('Mozambique',),
 ('Seychelles',),
 ('Chad',),
 ('Zimbabwe',),
 ('Malawi',),
 ('Le

In [22]:
cursor.execute("""
SELECT country, count(country) num_of_airports 
FROM airports
GROUP BY country
ORDER BY num_of_airports DESC""").fetchall()

[('United States', 1697),
 ('Canada', 435),
 ('Germany', 321),
 ('Australia', 263),
 ('Russia', 249),
 ('France', 233),
 ('China', 219),
 ('Brazil', 213),
 ('United Kingdom', 210),
 ('India', 140),
 ('Japan', 131),
 ('Indonesia', 114),
 ('South Africa', 103),
 ('Argentina', 103),
 ('Mexico', 94),
 ('Italy', 92),
 ('Sweden', 86),
 ('Spain', 84),
 ('Iran', 81),
 ('Turkey', 77),
 ('Colombia', 72),
 ('Norway', 69),
 ('Philippines', 64),
 ('Greece', 60),
 ('Venezuela', 58),
 ('Thailand', 58),
 ('Papua New Guinea', 57),
 ('New Zealand', 56),
 ('Kenya', 55),
 ('Switzerland', 53),
 ('Finland', 51),
 ('Malaysia', 47),
 ('Pakistan', 46),
 ('Madagascar', 45),
 ('Algeria', 44),
 ('Peru', 43),
 ('Burma', 43),
 ('Saudi Arabia', 42),
 ('Netherlands', 42),
 ('Maldives', 42),
 ('Portugal', 39),
 ('Ecuador', 38),
 ('Chile', 38),
 ('Bahamas', 37),
 ('Nepal', 36),
 ('French Polynesia', 36),
 ('Congo (Kinshasa)', 36),
 ('Poland', 34),
 ('Belgium', 34),
 ('Ukraine', 33),
 ('Taiwan', 32),
 ('Marshall Islands

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table. <br/>
Hint: Which column can serve as the link between the two tables? That is, which column in `airports` can serve as foreign key to `routes`?

In [51]:
cursor.execute("""
SELECT * 
FROM routes
JOIN airports
ON (airports.code = routes.source) 
""").fetchall()

[(0,
  '2B',
  '410',
  'AER',
  '2965',
  'KZN',
  '2990',
  None,
  '0',
  'CR2',
  2886,
  '2965',
  'Sochi',
  'Sochi',
  'Russia',
  'AER',
  'URSS',
  '43.449928',
  '39.956589',
  '89',
  '4',
  'N',
  'Europe/Moscow'),
 (1,
  '2B',
  '410',
  'ASF',
  '2966',
  'KZN',
  '2990',
  None,
  '0',
  'CR2',
  2887,
  '2966',
  'Astrakhan',
  'Astrakhan',
  'Russia',
  'ASF',
  'URWA',
  '46.283333',
  '48.006278',
  '-65',
  '4',
  'N',
  'Europe/Moscow'),
 (2,
  '2B',
  '410',
  'ASF',
  '2966',
  'MRV',
  '2962',
  None,
  '0',
  'CR2',
  2887,
  '2966',
  'Astrakhan',
  'Astrakhan',
  'Russia',
  'ASF',
  'URWA',
  '46.283333',
  '48.006278',
  '-65',
  '4',
  'N',
  'Europe/Moscow'),
 (3,
  '2B',
  '410',
  'CEK',
  '2968',
  'KZN',
  '2990',
  None,
  '0',
  'CR2',
  2889,
  '2968',
  'Balandino',
  'Chelyabinsk',
  'Russia',
  'CEK',
  'USCC',
  '55.305836',
  '61.503333',
  '769',
  '6',
  'N',
  'Asia/Yekaterinburg'),
 (4,
  '2B',
  '410',
  'CEK',
  '2968',
  'OVB',
  '4078'

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest numbers of active airlines?
- Which countries have the highest numbers of inactive airlines?
- What about airports by time zone?

In [52]:
cursor.execute(
"""
SELECT COUNT() num, country
FROM airlines
WHERE active='Y'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30
"""
).fetchall()

[(141, 'United States'),
 (72, 'Russia'),
 (40, 'United Kingdom'),
 (37, 'Germany'),
 (34, 'Canada'),
 (26, 'Australia'),
 (25, 'China'),
 (24, 'Spain'),
 (23, 'Brazil'),
 (22, 'France'),
 (19, 'Japan'),
 (18, 'Italy'),
 (17, 'Indonesia'),
 (17, 'India'),
 (16, 'Turkey'),
 (16, 'Thailand'),
 (15, 'Sweden'),
 (14, 'Switzerland'),
 (13, 'Ukraine'),
 (13, 'Portugal'),
 (12, 'Mexico'),
 (12, 'Finland'),
 (12, 'Egypt'),
 (12, 'Austria'),
 (11, 'Peru')]

In [53]:
cursor.execute(
"""
SELECT COUNT() num, country
FROM airlines
WHERE active='N'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30
"""
).fetchall()

[(939, 'United States'),
 (427, 'Mexico'),
 (367, 'United Kingdom'),
 (284, 'Canada'),
 (158, 'Russia'),
 (142, 'Spain'),
 (97, 'France'),
 (94, 'Germany'),
 (81, 'South Africa'),
 (80, 'Nigeria'),
 (76, 'Ukraine'),
 (73, 'Kazakhstan'),
 (72, 'Italy'),
 (67, 'Australia'),
 (55, 'Sweden'),
 (47, 'Sudan'),
 (46, 'Switzerland'),
 (45, 'China'),
 (43, 'Netherlands'),
 (38, 'Austria'),
 (36, 'Egypt'),
 (35, 'Chile'),
 (35, 'Brazil'),
 (35, 'Angola'),
 (34, 'Colombia'),
 (32, 'Thailand'),
 (32, 'Portugal'),
 (32, 'Mauritania'),
 (31, 'Indonesia'),
 (29, 'Kyrgyzstan')]

In [58]:
cursor.execute(
"""
SELECT  name, timezone
FROM airports
ORDER BY timezone
LIMIT 30
"""
).fetchall()

[('Abidjan Felix Houphouet Boigny Intl', 'Africa/Abidjan'),
 ('Bouake', 'Africa/Abidjan'),
 ('Daloa', 'Africa/Abidjan'),
 ('Korhogo', 'Africa/Abidjan'),
 ('Man', 'Africa/Abidjan'),
 ('San Pedro', 'Africa/Abidjan'),
 ('Yamoussoukro', 'Africa/Abidjan'),
 ('Kotoka Intl', 'Africa/Accra'),
 ('Tamale', 'Africa/Accra'),
 ('Wa', 'Africa/Accra'),
 ('Sunyani', 'Africa/Accra'),
 ('Takoradi', 'Africa/Accra'),
 ('Kumasi Airport', 'Africa/Accra'),
 ('Bole Intl', 'Africa/Addis_Ababa'),
 ('Lideta', 'Africa/Addis_Ababa'),
 ('Arba Minch', 'Africa/Addis_Ababa'),
 ('Axum', 'Africa/Addis_Ababa'),
 ('Bahir Dar', 'Africa/Addis_Ababa'),
 ('Dire Dawa Intl', 'Africa/Addis_Ababa'),
 ('Gambella', 'Africa/Addis_Ababa'),
 ('Gondar', 'Africa/Addis_Ababa'),
 ('Jimma', 'Africa/Addis_Ababa'),
 ('Lalibella', 'Africa/Addis_Ababa'),
 ('Makale', 'Africa/Addis_Ababa'),
 ('Asosa', 'Africa/Addis_Ababa'),
 ('Tippi Airport', 'Africa/Addis_Ababa'),
 ('Mizan Teferi Airport', 'Africa/Addis_Ababa'),
 ('Kabri Dehar Airport', 'Africa

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [59]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/\
views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [60]:
shelter_data[shelter_data['Name'] == "Max"].head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
43,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
289,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
378,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
924,A784016,Max,05/21/2019 03:13:00 PM,05/21/2019 03:13:00 PM,08/10/2018,Return to Owner,NaN,Dog,Intact Male,9 months,Staffordshire Mix,Brown Brindle/White
977,A681062,Max,06/12/2014 02:22:00 PM,06/12/2014 02:22:00 PM,06/11/2011,Return to Owner,NaN,Dog,Neutered Male,3 years,Jack Russell Terrier Mix,Brown/White


In [61]:
# With .query()

max_data = shelter_data.query('Name == "Max"')
max_data.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
43,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
289,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
378,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
924,A784016,Max,05/21/2019 03:13:00 PM,05/21/2019 03:13:00 PM,08/10/2018,Return to Owner,NaN,Dog,Intact Male,9 months,Staffordshire Mix,Brown Brindle/White
977,A681062,Max,06/12/2014 02:22:00 PM,06/12/2014 02:22:00 PM,06/11/2011,Return to Owner,NaN,Dog,Neutered Male,3 years,Jack Russell Terrier Mix,Brown/White


## Goal 6: Transfering from sqlite to pandas

`.read_sql_query()`

In [62]:
import pandas as pd

conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("SELECT * FROM airports LIMIT 50", conn)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
5,5,6,Wewak Intl,Wewak,Papua New Guinea,WWK,AYWK,-3.583828,143.669186,19,10,U,Pacific/Port_Moresby
6,6,7,Narsarsuaq,Narssarssuaq,Greenland,UAK,BGBW,61.160517,-45.425978,112,-3,E,America/Godthab
7,7,8,Nuuk,Godthaab,Greenland,GOH,BGGH,64.190922,-51.678064,283,-3,E,America/Godthab
8,8,9,Sondre Stromfjord,Sondrestrom,Greenland,SFJ,BGSF,67.016969,-50.689325,165,-3,E,America/Godthab
9,9,10,Thule Air Base,Thule,Greenland,THU,BGTL,76.531203,-68.703161,251,-4,E,America/Thule


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## On Your Own

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [63]:
# Getting Started

con = sqlite3.connect('Chinook_Sqlite.sqlite')


In [64]:
cursor2 = con.cursor()

In [65]:
cursor2.execute(
"""
SELECT tbl_name
FROM sqlite_master
WHERE type='table'
"""
)

In [66]:
cursor2.fetchall()

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

In [72]:
df =pd.read_sql_query("""
SELECT * 
FROM artist
ORDER BY name
""",con)
df

,ArtistId,Name
0,43,A Cor Do Som
1,1,AC/DC
2,230,Aaron Copland & London Symphony Orchestra
3,202,Aaron Goldberg
4,214,Academy of St. Martin in the Fields & Sir Nevi...
...,...,...
270,181,Xis
271,255,Yehudi Menuhin
272,212,Yo-Yo Ma
273,168,Youssou N'Dour
